In [1]:
!conda install -y gdown

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
!gdown --id 14mto87jQZLt4hTOcVV1H3wIyF1cvS4fS


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14mto87jQZLt4hTOcVV1H3wIyF1cvS4fS
To: /kaggle/working/glove.6B.50d.txt
100%|████████████████████████████████████████| 171M/171M [00:02<00:00, 84.1MB/s]


In [9]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
X_train = newsgroups_train.data
X_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [5]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/kaggle/working/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [6]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)

Found 179209 unique tokens.
(18846, 500)
Total 400000 word vectors.


In [10]:
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50):
    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, 20)


In [11]:
model_RCNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           8960500   
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 499, 256)          25856     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 249, 256)         0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 248, 256)          131328    
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 124, 256)         0         
 1D)                                                  

In [12]:
model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)


Epoch 1/15
89/89 - 164s - loss: 2.6803 - accuracy: 0.1121 - val_loss: 2.3751 - val_accuracy: 0.1775 - 164s/epoch - 2s/step
Epoch 2/15
89/89 - 157s - loss: 2.1796 - accuracy: 0.2033 - val_loss: 2.0958 - val_accuracy: 0.2369 - 157s/epoch - 2s/step
Epoch 3/15
89/89 - 153s - loss: 1.8462 - accuracy: 0.3006 - val_loss: 1.7612 - val_accuracy: 0.3299 - 153s/epoch - 2s/step
Epoch 4/15
89/89 - 161s - loss: 1.5602 - accuracy: 0.4002 - val_loss: 1.6437 - val_accuracy: 0.3910 - 161s/epoch - 2s/step
Epoch 5/15
89/89 - 159s - loss: 1.2325 - accuracy: 0.5258 - val_loss: 1.6303 - val_accuracy: 0.4553 - 159s/epoch - 2s/step
Epoch 6/15
89/89 - 164s - loss: 1.0774 - accuracy: 0.5893 - val_loss: 1.2491 - val_accuracy: 0.5542 - 164s/epoch - 2s/step
Epoch 7/15
89/89 - 158s - loss: 0.8888 - accuracy: 0.6665 - val_loss: 1.1723 - val_accuracy: 0.6005 - 158s/epoch - 2s/step
Epoch 8/15
89/89 - 161s - loss: 0.7604 - accuracy: 0.7218 - val_loss: 1.1212 - val_accuracy: 0.6433 - 161s/epoch - 2s/step
Epoch 9/15
89/89

In [13]:
predicted = model_RCNN.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))

236/236 [==============================] - 32s 133ms/step
              precision    recall  f1-score   support

           0       0.51      0.62      0.56       319
           1       0.48      0.70      0.57       389
           2       0.73      0.64      0.68       394
           3       0.54      0.73      0.62       392
           4       0.58      0.65      0.61       385
           5       0.76      0.66      0.71       395
           6       0.77      0.73      0.75       390
           7       0.68      0.83      0.75       396
           8       0.96      0.66      0.78       398
           9       0.86      0.84      0.85       397
          10       0.97      0.91      0.94       399
          11       0.90      0.80      0.84       396
          12       0.53      0.54      0.54       393
          13       0.89      0.80      0.84       396
          14       0.89      0.71      0.79       394
          15       0.80      0.81      0.81       398
          16       0.66